In [1078]:
# Install eli5, you can use any other model Interpretability libraries
!pip install eli5 -q

In [44]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import eli5
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [45]:
train = pd.read_csv('/home/fjaadari/Downloads/Train (4).csv')
test = pd.read_csv('/home/fjaadari/Downloads/Test (4).csv')
ss = pd.read_csv('/home/fjaadari/Downloads/SampleSubmission (5).csv')


## Identifying and Dropping Columns with High Percentage of Missing Values

In [46]:
percent_missing = train.isnull().sum() * 100/ len(train)
missing_value_train = pd.DataFrame({'column_name': train.columns,
                                  'percent_missing': percent_missing})
columns_to_drop = list(percent_missing[percent_missing >= 39].index)
missing_value_train.sort_values(by="percent_missing",ascending=True)


,column_name,percent_missing
child_id,child_id,0.000000
pre_covid,pre_covid,0.000000
sef_ind,sef_ind,0.000000
language_assessment,language_assessment,0.000000
hle_ind,hle_ind,0.000000
...,...,...
pra_class_space_small,pra_class_space_small,99.895166
pri_founderother,pri_founderother,99.906814
pri_clinic_travelother,pri_clinic_travelother,99.941759
pri_locationother,pri_locationother,99.941759


In [4]:
len(columns_to_drop)

609

## return back relevant features

In [47]:
columns_to_drop.remove("child_attends")
columns_to_drop.remove("child_attendance")
columns_to_drop.remove("pra_salary")








In [48]:
train = train.drop(columns = columns_to_drop)
test = test.drop(columns = columns_to_drop )
train.shape, test.shape



((8585, 73), (3680, 72))

In [49]:
data = pd.concat([train, test]).reset_index(drop=True)


## Feature Engineering Based of EDA 

In [51]:
data['child_dob'] = pd.to_datetime(data['child_dob'])
data['child_date'] = pd.to_datetime(data['child_date'])
data['diffday'] = (data['child_date'] - data['child_dob']).dt.days
data.loc[data["pri_fees_amount"] > 750, "child_grant"] = "No"
data.loc[data["language_match"] == 1.0, "child_grant"] = "Yes"
data.loc[data["pri_fees"] == "No", "child_grant"] = "Yes"
data.loc[data["language_child"] == "English", "child_grant"] = "No"
data.loc[data["teacher_social_total"] == 24, "child_grant"] = "Yes"
data.loc[data["teacher_social_total"] == 0, "child_grant"] = "No"
data['child_observe_total']=data["child_observe_total"].fillna(0)

class_scores = {'Almost never': 0, 'Sometimes': 1, 'Often': 2,'Almost always':3}

# create a function to calculate the total score for each row
def calculate_total_score(row):
    total_score = 0
    for col in ['child_observe_attentive', 'child_observe_concentrated', 'child_observe_diligent','child_observe_interested']:
        class_score = class_scores.get(row[col], 0)
        total_score += class_score
    return total_score

# apply the function to the three columns and assign the result to a new column
data['child_observe_total'] = data.apply(calculate_total_score, axis=1)










In [52]:
data = data.fillna(data.mode().iloc[0])
data=data.fillna(0)

In [53]:
data["id_enumerator"] = data["id_enumerator"].astype(str)
data["id_facility"] = data["id_facility"].astype(str)





In [54]:
from sklearn.preprocessing import OneHotEncoder
df=data.copy()
# create a sample dataframe with categorical variables
# loop over each categorical column in the dataframe
for col in df.select_dtypes(include=['object']):
    # check if the column has less than 150 unique values
    if len(df[col].unique()) < 300:
        
        encoder = OneHotEncoder(categories='auto', drop='first')
        one_hot = encoder.fit_transform(data[[col]])
        
        # create new column names with the prefix
        prefix = col + '_'
        new_cols = [prefix + str(i) for i in range(one_hot.shape[1])]
        
        # append the encoded columns to the original dataframe
        df_one_hot = pd.DataFrame(one_hot.toarray(), columns=new_cols)
        data = pd.concat([data, df_one_hot], axis=1)
        
        # drop the original column from the dataframe
        #data = data.drop(col, axis=1)


## dropping categorical columns

In [55]:
cat_cols = data.select_dtypes(include=['object', 'category']).columns

# drop categorical columns
data = data.drop(cat_cols, axis=1)


## StanderScaler

In [57]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

target_column = 'target'

# identify the numerical columns
numerical_columns = data.select_dtypes(include=["number"]).columns

# remove the target column from the list of numerical columns
numerical_columns = numerical_columns.drop(target_column)

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to the numerical columns
scaler.fit(data[numerical_columns])

# transform the numerical columns using the scaler
scaled_features = scaler.transform(data[numerical_columns])

# convert the scaled features array to a Pandas dataframe
scaled_features_df = pd.DataFrame(scaled_features, columns=numerical_columns)

# concatenate the target column with the scaled features dataframe
data = pd.concat([data[target_column], data[numerical_columns]], axis=1)


In [56]:
train = data[:train.shape[0]].reset_index(drop=True)
test = data[train.shape[0]:].reset_index(drop=True)


In [57]:
pd.set_option('display.max_columns', None)
numerical_columns = data.select_dtypes(include=["number"]).columns

l = numerical_columns.drop("target")


In [58]:
l=l.to_list()


In [59]:

y = train.target

X = train[l]


## Testing with LinearRegression after choosing best random_estate 254

In [60]:

X=X.fillna(0)
# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.09, random_state =1234)
# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check score
mean_squared_error(y_test, y_pred, squared=False)

9.235523093735928

In [20]:
model = LinearRegression()
model.fit(X, y)

# Make predictions
y_pred = model.predict(X_test)

# Check score
mean_squared_error(y_test, y_pred, squared=False)

8.83818968622995

In [65]:
# Make predictions on the test 
set
preds = model.predict(test[l].fillna(0))
#catboostpreds = best_cat.predict(test[l].fillna(0))


In [66]:
# Get the top five features for each predictions
from tqdm import tqdm

top_predictors = []

for i in tqdm(range(test.shape[0])):
  predictors = eli5.explain_prediction_df(estimator=model, doc=test[l].fillna(0).iloc[i], top = 15)
  top_predictors.append(predictors.feature.tolist())

100%|███████████████████████████████████████| 3680/3680 [00:54<00:00, 67.02it/s]


In [67]:
# Submission file preparation
test = pd.read_csv('/home/fjaadari/Downloads/Test (4).csv')

predictors = pd.DataFrame(top_predictors, columns = ['feature_' + str(i) for i in range(1,16)])
predictors['target'] = preds
predictors['child_id'] = test.child_id
predictors = predictors[ss.columns.tolist()]
predictors.head()

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,58.226456,<BIAS>,data_year,child_age,teacher_emotional_total,child_attendance,latitude,teacher_social_total,language_assessment_2,pri_fees_amount,id_ward,id_enumerator_31,language_match,id_dc_n,child_observe_concentrated_1,child_observe_total
1,ID_GQ6ONJ4FP,47.827996,<BIAS>,data_year,child_age,teacher_emotional_total,prov_best_2,child_attendance,latitude,teacher_social_total,pri_fees_amount,longitude,language_match,id_enumerator_186,id_prov_2,child_observe_diligent_1,child_observe_total
2,ID_YZ76CVRW3,49.452134,<BIAS>,data_year,child_age,teacher_emotional_total,id_dc_best_24,dc_best_12,pri_fees_amount,language_match,id_prov_4,id_dc_n,child_observe_interested_1,child_observe_attentive_1,child_observe_concentrated_1,child_observe_diligent_1,child_observe_total
3,ID_BNINCRXH8,68.639611,<BIAS>,data_year,child_age,dc_best_7,id_dc_best_0,teacher_emotional_total,id_enumerator_184,id_mn_n,child_attendance,prov_best_0,language_match,id_prov_7,id_dc_n,child_observe_interested_1,child_observe_total
4,ID_1U7GDTLRI,46.460987,<BIAS>,data_year,child_age,teacher_emotional_total,latitude,child_attendance,gps_ind_0,id_prov_n,id_dc_n,language_match,id_prov_7,child_observe_attentive_1,child_observe_concentrated_1,child_observe_diligent_1,child_observe_total


## Getting the Final result of LinearReg after Testing Multiple Seed Values and Test size

In [68]:
# Create csv to upload to Zindi for scoring
predictors.to_csv('LinearReg.csv', index = False)